# Preparing Linkset With Local Dumps


In [6]:
# https://dzone.com/articles/listing-a-directory-with-python
import os

TrainingFiles= filter(lambda x: x.endswith('_ok.csv'),  os.listdir(path)) 
 
for file in TrainingFiles : 
    print(file)
    

Thist2CGI_ICST_ok.csv
Thist2DBPEDIA_ok.csv
Thist2AGROVOC_ok.csv
Thist2GEMET_ok.csv
Thist2EUROVOC_ok.csv


## Getting a cleaned linkset

In [2]:
import pandas as pd
# It cleans the linkset providing subjects, objects, and kindOfLink

def cleaningLinkset(objname, path,  subname,):
    #path="data/" # path where to find data
    namesa=['sBT','sprefLabel','s','o','oprefLabel','oBT', 'KindOfLink'] #column names for training data frame
    # reading Thist2DBPEDIA_ok.csv
    df=pd.read_csv(path+ objname+'2'+subname+'_ok.csv',delimiter=";",names=namesa)
    #deleting row 0 which contains the old colums name
    df=df.drop([0])
    #THIST2DBPEDIA_df.head()
    df_linkeset=df[['s','o','KindOfLink']].drop_duplicates()
    #THIST2DBPEDIA_df_linkeset.head()
    return  df_linkeset


In [15]:
file="Thist2CGI_ICST_ok.csv"
file=file.replace('.csv','Clean.csv')
print(file)

Thist2CGI_ICST_okClean.csv


#### 1. Querying Thist's concepts involved in a link 

In [41]:
#variable 
path1='data/ToBeValidateRevised/'

In [49]:
#%%bash -s "$path" 
#java arq.sparql  --data $1/THIST2DBPEDIANEW.nt --data $1/datasets/ThIST20140804.ttl \
# --query $1/GenerateCSV4ValidationOnLocalDumps.sparql --results CSV  > $1/THISTTHIST2DBPEDIANEWsub.csv


In [49]:
import subprocess

def preparingSubjectTHIST(path):
    query="""prefix skos: <http://www.w3.org/2004/02/skos/core#> 
    prefix skosxl: <http://www.w3.org/2008/05/skos-xl#>
    PREFIX  :       <.> 

    select distinct  
    (GROUP_CONCAT(?sBroaderPrefLabel; separator=\"| \") as ?sBT)  
    (GROUP_CONCAT(?sNarrowerPrefLabel; separator=\"| \") as ?sNT) 
    (GROUP_CONCAT(?sRelatedPrefLabel; separator=\"| \") as ?sRT)  
    ?s 
    ?sPrefLabel 
    (GROUP_CONCAT(?sAltLabel ; separator=\"| \") as ?sAltLabels)  
    where { 
    ?s skos:exactMatch ?o.  
     filter regex(str(?s),\"^http://linkeddata.ge.imati.cnr.it/resource/ThIST/\",'i'). 
    # filter regex(str(?o),\"^SBJNS\",'i')   
       optional { ?s skos:prefLabel ?sPrefLabel. filter (lang(?sPrefLabel)='en')}.  
        optional { ?s skos:altLabel  ?sAltLabel . filter (lang(?sAltLabel)='en')}.  
       optional{ 
         ?s skos:broader ?sBroader. ?sBroader skos:prefLabel ?sBroaderPrefLabel. filter (lang(?sBroaderPrefLabel)='en') } .  
       optional{ 
         ?s skos:narrower ?sNarrower. ?sNarrower skos:prefLabel ?sNarrowerPrefLabel. filter (lang(?sNarrowerPrefLabel)='en') } .  
       optional{ 
         ?s skos:related ?sRelated. ?sRelated skos:prefLabel ?sRelatedPrefLabel. filter (lang(?sRelatedPrefLabel)='en') } .  
    } 
    group by ?s ?sPrefLabel 
    order by ?s """ 
    
    #let's create the query updated  for file
    #query=query.replace('SBJNS',datasetlist.iloc[index]['namespace'])
    text_file = open(path+"/tempquery.sparql", "w")
    text_file.write(query)
    text_file.close()
    
    command = 'java arq.sparql   --data ' + path+'ThIST20140804.ttl'  +' --query ' +path+'tempquery.sparql --results CSV  > ' +path+'THISTNEWsub.csv'
    print(command)

    #process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
    #output, error = process.communicate()
    #return 
    

In [50]:
#test
preparingSubjectTHIST( path1)

java arq.sparql   --data data/ToBeValidateRevised/ThIST20140804.ttl --query data/ToBeValidateRevised/tempquery.sparql --results CSV  > data/ToBeValidateRevised/THISTNEWsub.csv


In [44]:
#claning duplicates on sBT cells
#http://jonathansoma.com/lede/foundations/classes/pandas%20columns%20and%20functions/apply-a-function-to-every-row-in-a-pandas-dataframe/
def deletingDuplicates( df, columnName):
    for index, row in df.iterrows():
    #    print (row[columnName])
        # Let iterate for every row in the DF
        a= map(str.strip, df.iloc[index][columnName].split('|'))
        mylist = list(dict.fromkeys(a))
        s='| '.join(str(f) for f in mylist)
        df.at[index,columnName]=s
    return df

In [45]:

import pandas as pd
def cleaning(path, file):
    # reading Thist2DBPEDIA_ok.csv
    df=pd.read_csv(path+file,delimiter=",")
    #THIST_dfa.head(2)
    df=df.fillna("")

    df=deletingDuplicates(df,'sBT')
    df=deletingDuplicates(df,'sNT')
    df=deletingDuplicates(df,'sRT')
    df=deletingDuplicates(df,'sAltLabels')

    df.to_csv(path+file.replace('.csv','Clean.csv'), sep=',', index=False)


In [51]:
cleaning(path1,'THISTNEWsub.csv')

In [53]:
# TODO merge THIST2DBPEDIA_df_linkeset with NEWTHIST
import pandas as pd
def  merging(linkset, datasetfile, field):

    d=pd.read_csv(path+'/'+datasetfile,delimiter=",")

    NEWTHIST=NEWTHIST.fillna("")
    #print(NEWTHIST)
    linkeset=linkeset.fillna("")
    #print(THIST2DBPEDIA_df_linkeset)
    #linkeset.describe()

    result= pd.merge(linkeset, dataset, on=[field])
    return result


#### 2. Querying Differet Linkset Objects 

In [92]:
path="data/ToBeValidateRevised/" # path where to find data
import pandas as pd

datasetList=pd.DataFrame([ 
        ['Thist','http://linkeddata.ge.imati.cnr.it/resource/ThIST/'],
        ['AGROVOC','http://aims.fao.org/aos/agrovoc', 'AGROVOC20140722.ttl'],
        ['GEMET','http://www.eionet.europa.eu/gemet/','GEMET20140608.ttl'],
        ['EUROVOC','http://eurovoc.europa.eu/','EUROVOC20121215.ttl']], columns= ['name', 'namespace', 'file' ], index=[1,2,3,4])

#for name, namespace in datasetList:
    
datasetList.iloc[1]['file']

'AGROVOC20140722.ttl'

In [100]:
import subprocess

def preparingObject(index, path, datasetlist):
    query="""prefix skos: <http://www.w3.org/2004/02/skos/core#> 
    prefix skosxl: <http://www.w3.org/2008/05/skos-xl#>
    PREFIX  :       <.> 

    select distinct  
    (GROUP_CONCAT(?oBroaderPrefLabel; separator=\"| \") as ?oBT)  
    (GROUP_CONCAT(?oNarrowerPrefLabel; separator=\"| \") as ?oNT) 
    (GROUP_CONCAT(?oRelatedPrefLabel; separator=\"| \") as ?oRT)  
    ?o 
    ?oPrefLabel 
    (GROUP_CONCAT(?oAltLabel ; separator=\"| \") as ?oAltLabels)  
    where { 
    ?s skos:exactMatch ?o.  
     filter regex(str(?s),\"^http://linkeddata.ge.imati.cnr.it/resource/ThIST/\",'i'). 
     filter regex(str(?o),\"^SBJNS\",'i')   
       optional { ?o skos:prefLabel ?oPrefLabel. filter (lang(?oPrefLabel)='en')}.  
        optional { ?o skos:altLabel  ?oAltLabel . filter (lang(?oAltLabel)='en')}.  
       optional{ 
         ?o skos:broader ?oBroader. ?oBroader skos:prefLabel ?oBroaderPrefLabel. filter (lang(?oBroaderPrefLabel)='en') } .  
       optional{ 
         ?o skos:narrower ?oNarrower. ?oNarrower skos:prefLabel ?oNarrowerPrefLabel. filter (lang(?oNarrowerPrefLabel)='en') } .  
       optional{ 
         ?o skos:related ?oRelated. ?oRelated skos:prefLabel ?oRelatedPrefLabel. filter (lang(?oRelatedPrefLabel)='en') } .  
    } 
    group by ?o ?oPrefLabel 
    order by ?o """ 
    
    #let's create the query updated  for file
    query=query.replace('SBJNS',datasetlist.iloc[index]['namespace'])
    text_file = open(path+"/tempquery.sparql", "w")
    text_file.write(query)
    text_file.close()
    
    command = 'java -Xmx6072M arq.sparql   --data ' + path+'ThIST20140804.ttl --data ' + path+'datasets/'+datasetlist.iloc[index]['file']  +' --query ' +path+'tempquery.sparql --results CSV  > ' +path+datasetlist.iloc[index]['file'].replace('.ttl','sub.csv')
    print(command)

    process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
    process.wait()
    output, error = process.communicate()
    #return 
    

In [107]:
# TODO merge THIST2DBPEDIA_df_linkeset with NEWTHIST
import pandas as pdL 
def RemovingDuplicatesOnBTNTRT(index, path,  datasetlist):
    dSub=pd.read_csv(path+datasetlist.iloc[index]['file'].replace('.ttl','sub.csv'),delimiter=",")

    dSub=dSub.fillna("")
    #print(NEWTHIST)
    dSub=deletingDuplicates(dSub,'oBT')
    dSub=deletingDuplicates(dSub,'oNT')
    dSub=deletingDuplicates(dSub,'oRT')
    dSub=deletingDuplicates(dSub,'oAltLabels')
    print('writing '+path+(datasetlist.iloc[index]['file']).replace('.ttl','Clean.csv'))
    dSub.to_csv(path+datasetlist.iloc[index]['file'].replace('.ttl','Clean.csv'), sep=',', index=False)
    



In [108]:
#MAnually generating the command to query, let's print the command and exect them manually on the shell
preparingObject(3, path, datasetList)

java -Xmx6072M arq.sparql   --data data/ToBeValidateRevised/ThIST20140804.ttl --data data/ToBeValidateRevised/datasets/EUROVOC20121215.ttl --query data/ToBeValidateRevised/tempquery.sparql --results CSV  > data/ToBeValidateRevised/EUROVOC20121215sub.csv


In [109]:
#It removes duplicates in all except for dbpedia.
print(len(datasetList))

l=range(1, len(datasetList))
for i in l :
    #print('preparing '+ datasetList.iloc[i]['name'])           
    #preparingObject(i,path, datasetList)
    print('RemovingDuplicatesOnBTNTRT '+ datasetList.iloc[i]['name'])  
    RemovingDuplicatesOnBTNTRT(i, path, datasetList)
   



4
RemovingDuplicatesOnBTNTRT AGROVOC
writing data/ToBeValidateRevised/AGROVOC20140722Clean.csv
RemovingDuplicatesOnBTNTRT GEMET
writing data/ToBeValidateRevised/GEMET20140608Clean.csv
RemovingDuplicatesOnBTNTRT EUROVOC
writing data/ToBeValidateRevised/EUROVOC20121215Clean.csv


In [226]:
creatingEnrichedLinkes(path, datasetList )

reading data/ToBeValidateRevised/THISTNEWsubClean.csv
reading data/Thist2AGROVOC_ok.csv
reading data/ToBeValidateRevised/AGROVOC20140722Clean.csv
**** object
                                       oBT  \
0                                 Pinaceae   
1                                 symptoms   
2                   Musculoskeletal system   
3                              enterprises   
4                     Functional disorders   
5                       Genetic soil types   
6                                  Borates   
7                          Inorganic acids   
8                          South East Asia   
9                               Semimetals   
10                                Animalia   
11                 biology| plant sciences   
12                         Southern Africa   
13                               Packaging   
14                               Sociology   
15                              Ruminantia   
16                                Animalia   
17            

writing enriched linkset Thist2EUROVOCEnrichedLinkeset.csv
